In [ ]:
import subprocess, os, time
from sqlalchemy import (Column, ForeignKey, Integer, MetaData, String, Table,
                        create_engine)
from sqlalchemy.orm import (Mapped, declarative_base, mapped_column,
                            relationship, sessionmaker)
from models import *

# Create a session
home_directory = os.path.expanduser('~')
engine = create_engine(f'sqlite:///{home_directory}/Tools/Scenic/scenic_database/scenario_database.db', echo=False)
Session = sessionmaker(bind=engine)
session = Session()

# List of script paths
scripts = session.query(TCEnhancedConcreteScenario).all()

def gen():
    for script in scripts:
        yield script
s = gen()

In [ ]:

script = str(next(s).path).replace(".scenic",".py").replace("~",home_directory)
# Run the script
subprocess.run(['code', script.replace(".py",".scenic")])
subprocess.run(['code', script])

print("All scripts have been executed.")


In [2]:
import subprocess, os, time, re
from sqlalchemy import (Column, ForeignKey, Integer, MetaData, String, Table,
                        create_engine)
from sqlalchemy.orm import (Mapped, declarative_base, mapped_column,
                            relationship, sessionmaker)
from models import *

# Create a session
home_directory = os.path.expanduser('~')
engine = create_engine(f'sqlite:///{home_directory}/Tools/Scenic/scenic_database/scenario_database.db', echo=False)
Session = sessionmaker(bind=engine)
session = Session()

# List of script paths
scripts = session.query(TCEnhancedConcreteScenario).all()
print(f"{len(scripts)} test cases")

flag = False
flag_file = ""
for idx, script in enumerate(scripts[83:]):
    script = str(script.path).replace(".scenic",".py").replace("~",home_directory)
    if flag_file == "":
        print(f"{idx}. Testing: ",script)
    elif script != flag_file and flag == False:
        print("Pass: ",script)
        continue
    else:
        flag = True
        print(f"{idx}. Testing: ",script)

    try:
        # Run the script
        result = subprocess.run(['/home/weidong/miniconda3/envs/scenic_related/bin/python', script], check=True, capture_output=True, text=True)
        # result = subprocess.run(['/bin/python3', script], check=True, capture_output=True, text=True)
        
        # pattern = re.compile(r'\*\*\*result:')
        # matches = pattern.findall(result.stdout)
        
        # if matches:
        #     print(f"Output of {script}:")
        #     for match in matches:
        #         print(match)
        # else:
        #     print(f"No result given in {script}.")

        print(str(result.stdout).split("***results:")[-1])
        if result.stderr:
            print(f"Error output:\n{result.stderr}")
            
    except subprocess.CalledProcessError as e:
        print(f"Error running {script}: {e}")
        print(f"Return code: {e.returncode}")
        print(f"Output: {e.output}")
        print(f"Error output: {e.stderr}")

192 test cases
0. Testing:  /home/weidong/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf1/nl1/nc1/tcc/Multiple_vehicles_cut_in_to_ego_path_in_a_short_time/tcc5/tcc5.py
apollo_dev_weidong
No process found occupying port 8000.
No process found occupying port 8000.
No process found occupying port 8000.
Ego position: (-2.049259901046753, -59.959224700927734, 0.6000000238418579, 180.0, 90.01206970214844, 180.0)
Ego destination: (-2.0111985206604004, -184.95921325683594, 0.0)
Running command: git checkout -- modules/common/data/global_flagfile.txt in directory /home/weidong/Tools/apollo/
Subprocess executed successfully
Running command: docker exec -u weidong apollo_dev_weidong ./scripts/bootstrap.sh restart in directory /home/weidong
Subprocess executed successfully
Running command: tmux new-session -d -s bridge_session docker exec -w /apollo/modules/carla_bridge/ -u weidong apollo_dev_weidong sh -c export PYTHONPATH=$PYTHONPATH:/apollo/bazel-bin/cyber/python/internal &&          

KeyboardInterrupt: 

15. Testing:  /home/weidonghu/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf2/nl4/nc8/tcc/Cyclist_crosses_from_the_roadside/tcc58/tcc58.py
 {'collision': False, 'ego_reach_destination': False, 'ego_speed_limit': True, 'ego_lane_keeping': True, 'conclusion': False}

### Delete all rows in a table

In [ ]:
import subprocess, os, time
from sqlalchemy import (Column, ForeignKey, Integer, MetaData, String, Table,
                        create_engine, delete)
from sqlalchemy.orm import (Mapped, declarative_base, mapped_column,
                            relationship, sessionmaker)
from models import *

# Create a session
home_directory = os.path.expanduser('~')
engine = create_engine(f'sqlite:///{home_directory}/Tools/Scenic/scenic_database/scenario_database.db', echo=False)
Session = sessionmaker(bind=engine)
session = Session()

# Delete all rows from the table
session.execute(delete(TestResultHazardousBehavior))
session.execute(delete(TestResult))

# Commit the transaction
session.commit()

# Close the session
session.close()

### find all linked scenarios

In [ ]:
import subprocess, os, time
from sqlalchemy import (Column, ForeignKey, Integer, MetaData, String, Table,
                        create_engine, delete)
from sqlalchemy.orm import (Mapped, declarative_base, mapped_column,
                            relationship, sessionmaker)
from models import *


home_dir = os.path.expanduser('~')
# Create a session
home_directory = os.path.expanduser('~')
engine = create_engine(f'sqlite:///{home_directory}/Tools/Scenic/scenic_database/scenario_database.db', echo=False)
Session = sessionmaker(bind=engine)
session = Session()

tcc = "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf2/nl4/nc8/tcc/Cyclist_crosses_from_the_roadside/tcc57/tcc57.scenic"

logical_scenario = session.query(TCEnhancedLogicalScenario).\
                            join(TCEnhancedConcreteScenario).\
                            filter(TCEnhancedConcreteScenario.path == tcc).first()


concrete_scenarios = session.query(TCEnhancedConcreteScenario).\
                            join(TCEnhancedLogicalScenario).\
                            filter(TCEnhancedLogicalScenario.path == logical_scenario.path).all()
concrete_scenarios.append(logical_scenario)

for tcc in concrete_scenarios:
    path = str(tcc.path).replace("~",home_dir)
    if os.path.exists(path):
        print(path)
        time.sleep(0.1)
        subprocess.run(['code', path])
